In [ ]:
from pocket_cube.cube import Cube
from pocket_cube.cube import Move
import tests
import numpy as np
from heapq import heappush, heappop

%matplotlib notebook


 # Tests

In [ ]:
from typing import Callable

test_list = [tests.case1, tests.case2, tests.case3, tests.case4]
test_list = list(map(lambda t: list(map(Move.from_str, t.split(" "))), test_list))
def test(algorithm: Callable[[Cube], list[Move]], tests: list[list[Move]]) -> bool:
    for idx, test in enumerate(tests):
        cube: Cube = Cube(test)
        path: list[Move] = algorithm(cube)
        for move in path:
            cube = cube.move(move)
        if not is_solved(cube):
            print(f"Test {idx} failed")
        else:
            print(f"Test {idx} passed")

 # A*

In [ ]:
FrontierItem = tuple[int, str, Cube]
DiscoveredDict = dict[str, tuple[str, Move, int]]

In [ ]:
def is_solved(cube: Cube) -> bool:
    for i in range(len(cube.state)):
        if cube.state[i] != cube.goal_state[i]:
            return False
    return True

In [ ]:
def get_neighbors(cube: Cube) -> list[tuple[Cube, Move]]:
    return [(cube.move(move), move) for move in Move]

In [ ]:
def heuristic(cube: Cube) -> int:
    return np.sum(cube.state != cube.goal_state)

In [ ]:
def get_path(cube_hash: str, discovered: DiscoveredDict) -> list[Move]:
    path: list[Move] = []
    currentNode = discovered[cube_hash]
    while currentNode[0] is not None:
        path.append(currentNode[1])
        currentNode = discovered[currentNode[0]]
    path.reverse()
    return path

In [ ]:
def a_star(cube: Cube) -> list[Move]:
    # initialize with cube
    frontier: list[FrontierItem] = []
    heappush(frontier, (0 + heuristic(cube), cube.hash(), cube.clone()))
    discovered: DiscoveredDict = {cube.hash(): (None, None, 0)}
    # search
    while frontier:
        currentCube: Cube = heappop(frontier)[2]
        if is_solved(currentCube):
            break
        for (neighbor, move) in get_neighbors(currentCube):
            score: int = discovered[currentCube.hash()][2] + 1
            if neighbor.hash() not in discovered or score < discovered[neighbor.hash()][2]:
                discovered[neighbor.hash()] = (currentCube.hash(), move, score)
                node: FrontierItem = (score + heuristic(neighbor), neighbor.hash(), neighbor.clone())
                heappush(frontier, node)
    # get path
    return get_path(currentCube.hash(), discovered)

 # Test A*

In [ ]:
test(a_star, test_list)

 # Bidirectional BFS

In [ ]:
def met_in_the_middle(cubes1: DiscoveredDict, cubes2: DiscoveredDict) -> str:
    for key in cubes1:
        if key in cubes2:
            return key
    return None

In [ ]:
from collections import deque

def bidirectional_bfs(cube: Cube) -> list[Move]:
    frontiers: list[deque[Cube]] = [deque(), deque()]
    frontiers[0].append(cube)

    solved_cube = cube.clone()
    solved_cube.state = solved_cube.goal_state
    frontiers[1].append(solved_cube)
    discovereds: list[DiscoveredDict] = [{cube.hash(): (None, None, 0)}, {solved_cube.hash(): (None, None, 0)}]

    while frontiers[0] and frontiers[1]:
        met_cube_key: str = met_in_the_middle(discovereds[0], discovereds[1])
        if met_cube_key is not None:
            break
        currentCubes: tuple[Cube] = (frontiers[0].popleft(), frontiers[1].popleft())
        for i in range(2):
            for (neighbor, move) in get_neighbors(currentCubes[i]):
                score: int = discovereds[i][currentCubes[i].hash()][2] + 1
                if neighbor.hash() not in discovereds[i] or score < discovereds[i][neighbor.hash()][2]:
                    discovereds[i][neighbor.hash()] = (currentCubes[i].hash(), move, score)
                    frontiers[i].append(neighbor)
    path1: list[Move] = get_path(met_cube_key, discovereds[0])
    path2: list[Move] = get_path(met_cube_key, discovereds[1])
    path2.reverse()
    path2 = list(map(Move.opposite, path2))
    return path1 + path2

 # Test Bidirectional BFS

In [ ]:
test(bidirectional_bfs, test_list)

 # MTCS with UCB

In [ ]:
N = "N"
Q = "Q"
PARENT = "PARENT"
MOVE = "MOVE"
CHILDREN = "CHILDREN"
Node = dict[int, int, Cube, dict[Move, Cube]]
def init_node(parent = None) -> Node:
    return {N: 0, Q: 0, PARENT: parent, CHILDREN: {}}

In [ ]:
from math import sqrt, log

def select_move(node: Node, c) -> Move:
    N_node = node[N]
    max_move: Move = None
    max_expr: float = float('-inf')
    for move in node[CHILDREN]:
        child = node[CHILDREN][move]
        expr = child[Q] / child[N] + c * sqrt(log(N_node) / child[N])
        if expr > max_expr:
            max_expr = expr
            max_move = move
    return max_move

In [ ]:
def hamming_heuristic(cube: Cube) -> int:
    return np.sum(cube.state == cube.goal_state)

In [ ]:
def manhattan_heuristic2(cube: Cube) -> int:
    return 0

In [ ]:
from random import choice

def mcts(cube0: Cube, budget: int, tree: Node, cp: float, heuristic: Callable[[Cube], int]) -> Node:
    if not tree:
        tree = init_node()
    for _ in range(budget):
        cube = cube0
        node = tree
        # go down the tree until a final state or an unexplored move is found
        while not is_solved(cube) and not any([move not in node[CHILDREN] for move in Move]):
            move: Move = select_move(node, cp)
            cube = cube.move(move)
            node = node[CHILDREN][move]
        # if node is not final and not every move has been explored, create a new node
        if not is_solved(cube):
            new_node: Node = init_node(node)
            move: Move = choice([move for move in Move if move not in node[CHILDREN]])
            node[CHILDREN][move] = new_node
            cube = cube.move(move)
            node = new_node
        # simulate a random game
        max_moves: int = 14
        max_h: int = 0
        while not is_solved(cube) and max_moves > 0:
            new_node: Node = init_node(node)
            move: Move = choice([move for move in Move])
            node[CHILDREN][move] = new_node
            cube = cube.move(move)
            node = new_node
            max_h = max(max_h, heuristic(cube))
            max_moves -= 1
        while node:
            node[N] += 1
            node[Q] += max_h
            node = node[PARENT]
    return tree

def play_mtcs(cube: Cube, budget: int, cp: float, heuristic: Callable[[Cube], int]) -> list[Move]:
    tree: Node = mcts(cube, budget, None, cp, heuristic)
    node: Node = tree
    path: list[Move] = []
    while node and node[CHILDREN]:
        move: Move = select_move(node, 0)
        node = node[CHILDREN][move]
        path.append(move)
    return path

 # Test MTCS